<a href="https://colab.research.google.com/github/drppdrpp/drppdrpp.github.io/blob/master/%E6%A0%AA%E4%BE%A1%E6%97%A5%E8%B6%B3%E6%AF%94%E8%BC%83%E6%A4%9C%E7%B4%A2_stooq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [120]:
#@title ① import関連
import pandas as pd
from pandas_datareader.stooq import StooqDailyReader
from datetime import datetime as dt
from datetime import date, datetime


In [ ]:
#@title ② 実行する (年、月、日、銘柄コードを入力) 　※api上、最新のデータは一昨日までしかない { display-mode: "code" }
# https://ja.visual-foxpro-programmer.com/how-can-i-hide-cell-contents-google-colaboratory
# 日付22,2,16 と銘柄コード3688として各々変数へ格納する
input_var = input('▤始▤ 年、月、日、銘柄コードをつなげて、10桁入力してください (例: 2202163688) → ')
input_end_var = input('▤終▤ 年、月、日をつなげて、 6桁入力してください (例: 220217)　　　　　　     → ')

# 銘柄コードをスライス
input_code = input_var[6:]
# 日付をスライス
input_date = input_var[:6]
# 三桁目が0なら0を
if input_date[2:3] == '0':
  year = int('20' + input_date[:2])
  month = int(input_date[3:4])
  day = int(input_date[4:])
else:
  year = int('20' + input_date[:2])
  month = int(input_date[2:4])
  day = int(input_date[4:])

# 日付をスライス
input_end_date = input_end_var[:6]
# 三桁目が0なら0を
if input_end_date[2:3] == '0':
  end_year = int('20' + input_end_date[:2])
  end_month = int(input_end_date[3:4])
  end_day = int(input_end_date[4:])
else:
  end_year = int('20' + input_end_date[:2])
  end_month = int(input_end_date[2:4])
  end_day = int(input_end_date[4:])

start = datetime(year, month, day)
end = datetime(end_year,end_month,  end_day)
brand = input_code + '.JP'

stooq = StooqDailyReader(brand, start=start, end=end)
data = stooq.read()  # pandas.core.frame.DataFrame
data = data.sort_values('Date', ascending=True)

# 注意：指数表記になってしまった場合、桁がe+として省略されてしまう以下で変更
# pd.options.display.float_format = '{:.2f}'.format #小数点を二桁までに設定 https://note.nkmk.me/python-pandas-option-display/
data['始終%'] = (data['Close'] - data['Open']) / data['Open'] * 100
data['初日始値から毎終値%'] = (data['Close'] - data.loc[start,'Open']) / data.loc[start,'Open'] * 100
print(data)

print('----------------------------------------------------------------------------------------')

#全何日なのか
count_all_days = end - start
all_days_percent = (data.loc[end,'Open'] - data.loc[start,'Open']) /  data.loc[start,'Open'] * 100
print(f'初日の始値から最終日の始値まで {all_days_percent:.2f}% 初日を0日として最終日を入れて 全{count_all_days.days}日(土日祝含む)')

#初日始値とcloseの最大が何日目なのか
max_after_day = data['Close'].idxmax()
max_after_day_in_format = max_after_day.strftime('%Y/%m/%d')
max_after_days = max_after_day - start
max_percent = data['初日始値から毎終値%'].max()
print(f'初日の始値から各終値の最高値   {max_percent:.2f}% 初日を0日として {max_after_days.days}日後({max_after_day_in_format})')

#初日始値とcloseの最低が何日目なのか
min_after_day = data['Close'].idxmin()
min_after_day_in_format = min_after_day.strftime('%Y/%m/%d')
min_after_days = min_after_day - start
min_percent = data['初日始値から毎終値%'].min()
print(f'初日の始値から各終値の最低値   {min_percent:.2f}% 初日を0日として {min_after_days.days}日後({min_after_day_in_format})')

print('----------------------------------------------------------------------------------------')

In [ ]:
#@title オプション: ローソクの出力
import numpy as np
import plotly.graph_objects as go

def plot_chart(df):
    x = np.arange(df.shape[0])

    interval = 20
    vals = np.arange(df.shape[0],step=interval)
    labels = list(df[::interval].index)
    
    fig = go.Figure(
            data=[go.Candlestick(
                    x=x,
                    open=df['Open'],
                    high=df['High'],
                    low=df['Low'],
                    close=df['Close'],
                    increasing_line_color= 'red', decreasing_line_color= 'blue')
                 ],
            layout = go.Layout(
                    xaxis = dict(
                        tickvals = vals,
                        ticktext = labels,
                        tickangle=-45 
                    ),
            )
    )
    fig.show()

plot_chart(data)